In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)

In [2]:
pitchers = pd.read_csv('all_teams_pitchers.csv')
batters = pd.read_csv('all_teams_hitters.csv')

In [12]:
p = pitchers[['Player', 'Team', 'Pos', 'Payroll Salary Adjusted', 'WAR']]

In [13]:
b = batters[['Player', 'Team', 'Pos', 'Payroll Salary Adjusted', 'WAR']]
df = pd.concat([p, b]).reset_index(drop=True)

In [41]:
b1 = batters.groupby('Team').agg({'Payroll Salary Adjusted': 'sum', 'WAR': 'sum'})

In [44]:
b1["Cost per War"] = b1["Payroll Salary Adjusted"] / b1["WAR"]

In [45]:
b1

,Payroll Salary Adjusted,WAR,Cost per War
Team,,,
ATL,76193094,17.8,4.280511e+06
AZ,67413434,33.7,2.000399e+06
BAL,35153075,30.4,1.156351e+06
BOS,55083163,22.2,2.481224e+06
CHC,115428170,25.3,4.562378e+06
CHW,65743680,-2.1,-3.130651e+07
CIN,19287050,7.1,2.716486e+06
CLE,40478197,20.8,1.946067e+06
COL,38466351,13.2,2.914117e+06


In [15]:
war_team = df['Payroll Salary Adjusted'].sum() / df['WAR'].sum()

In [28]:
team_agg = df.groupby('Team').agg({'Payroll Salary Adjusted':'sum', 'WAR':'sum'})

In [34]:
team_agg['Cost per War'] = team_agg['Payroll Salary Adjusted'] / team_agg['WAR']

In [32]:
team_agg['Cost per War'] = team_agg['Cost per War'] / 1000000

In [35]:
team_agg

,Payroll Salary Adjusted,WAR,Cost per War
Team,,,
ATL,166338167,45.7,3.639785e+06
AZ,134502970,42.5,3.164776e+06
BAL,65982048,40.5,1.629186e+06
BOS,77327289,33.5,2.308277e+06
CHC,177805468,36.1,4.925359e+06
CHW,74306691,10.8,6.880249e+06
CIN,54538311,24.2,2.253649e+06
CLE,50885032,39.4,1.291498e+06
COL,79534485,19.3,4.120958e+06
